In [ ]:
import pandas as pd
import numpy as np
import os
import time
import datetime
from openai import OpenAI
import pymssql
import re
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from threading import Thread
import functools
from tableone import TableOne
from scipy import stats
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu as mwu
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from lifelines.statistics import multivariate_logrank_test
from sksurv.nonparametric import kaplan_meier_estimator
from sklearn.metrics import roc_curve, auc
import random
from threading import Thread
import functools
import google.generativeai as genai

conn = pymssql.connect(host=, database=, charset=) # enter pymssql connection info
genai.configure(api_key="") # enter google cloud api key

In [ ]:
generation_config = {
  "temperature": 0,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

In [ ]:
rand_df_temp=pd.read_csv('20241018_gptderived_FSRP_testdf.csv')
rand_df_temp=rand_df_temp.iloc[:20000]
rand_df=rand_df_temp.copy()
for i in range(9):
    rand_df=pd.concat([rand_df,rand_df_temp])
rand_df.reset_index(inplace=True,drop=True)
rand_df['eid']=range(200000)

In [ ]:
usercontent_global1='''Estimate the risk (in percentages) of developing stroke within 10 years for the person below.
'''
usercontent_global2='''

Please answer exactly in the format below, without blank lines, and no further information or answer is required.
Risk percentage=(in percentages, round to one decimal place)'''

input_text=rand_df['input_text']
eid=rand_df['eid']

def timeout(timeout):
    def deco(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            res = [Exception('function [%s] timeout [%s seconds] exceeded!' % (func.__name__, timeout))]
            def newFunc():
                try:
                    res[0] = func(*args, **kwargs)
                except Exception as e:
                    res[0] = e
            t = Thread(target=newFunc)
            t.daemon = True
            try:
                t.start()
                t.join(timeout)
            except Exception as je:
                print ('error starting thread')
                raise je
            ret = res[0]
            #if isinstance(ret, BaseException):
            #    raise ret
            return ret
        return wrapper
    return deco

@timeout(0.1)
def ChatGPT_main(i,temper,responsenum):
    if input_text[i]!='':
        try:
            usercontent=usercontent_global1
            usercontent+=input_text[i]
            usercontent+=usercontent_global2

            model = genai.GenerativeModel(
              model_name="gemini-1.5-pro-002",
              generation_config=generation_config,
            )

            chat_session = model.start_chat(
              history=[
              ]
            )
            
            response = chat_session.send_message(usercontent)
            responsetext=response.text
            #print(responsetext)
            
            for j in range(1):
            #print(aa)
                temp1='blank'
                temp2=responsetext.split('=')[1]
                temp3='blank'
                temp4='blank'
                temp5='blank'
                
                conn = pymssql.connect(host=, database=, charset=) # enter pymssql connection info
                with conn:
                    with conn.cursor() as cur:
                        cur.execute(sql, (str(eid[i]),temp1,temp2,temp3,temp4,temp5))
                        conn.commit()

        except:
            temp1=np.nan
            temp2=np.nan
            temp3=np.nan
            temp4=np.nan
            temp5=np.nan
            
    else:
        temp1=np.nan
        temp2=np.nan
        temp3=np.nan
        temp4=np.nan
        temp5=np.nan
        


In [ ]:
responsenumnumnum=1
for experimentnum in range(1,2):
    print('=========================================================')
    print('experimentnum',experimentnum)
    input_text=rand_df['input_text']
    ############################
    table_name="20241029_GEMINI_stroke1"
    iterations=1
    upto=200000
    ############################
    for temper in [0.0]:
        time1=datetime.datetime.now()
        while True:
            toggle=0
            for i in range(iterations):   
                newtablaname=table_name + '_' + str(int(experimentnum)) + '_' + str(i)
                try:
                    sql_createtable="CREATE TABLE [" + newtablaname +"""] 
                    (
                        eid    NVARCHAR(20),
                        system    NVARCHAR(max),
                        score      NVARCHAR(max),
                        category     NVARCHAR(max) ,
                        framingham  NVARCHAR(20),
                        ACC_AHA  NVARCHAR(20)
                    )

                    """
                    conn = pymssql.connect(host=, database=, charset=) # enter pymssql connection info
                    with conn:
                        with conn.cursor() as cur:
                            cur.execute(sql_createtable)
                            conn.commit()
                    time.sleep(1)  
                except:
                    pass

                sql_statement="select * from ["+ newtablaname + "]"
                conn = pymssql.connect(host=, database=, charset=) # enter pymssql connection info
                datasql = pd.read_sql(sql=sql_statement, con=conn)
                datasql=datasql.astype({'eid':int})
                cnt_df=datasql.groupby('eid')['score'].count()
                tempcount=0
                for j in range(upto):
                    try:
                        cnt=cnt_df[j]
                        if cnt<responsenumnumnum:
                            addcnt=int(responsenumnumnum-cnt)
                            sql = "INSERT INTO [" + newtablaname+"] (eid, system,score,category,framingham,ACC_AHA) VALUES (%s, %s, %s, %s, %s, %s)"
                            ChatGPT_main(j,temper,addcnt)
                            tempcount+=1

                    except:
                        sql = "INSERT INTO [" + newtablaname+"] (eid, system,score,category,framingham,ACC_AHA) VALUES (%s, %s, %s, %s, %s, %s)"
                        ChatGPT_main(j,temper,responsenumnumnum)
                        tempcount+=1
                    if j%100==0:
                        print(j)

                if tempcount==0:
                    toggle=1
            if toggle==1:
                break
            time.sleep(20)

        time.sleep(20)


        time2=datetime.datetime.now()
        print(time2-time1)
        #print('except_else_num = ',except_else_num)

In [ ]:
table_name="20241029_GEMINI_stroke1"

In [ ]:
gpt_cindex=[]
validnum=[]
for experimentnum in range(1,2):
    print('=========================================================')
    print('experimentnum',experimentnum)
    #rand_df=pd.read_csv('20241023_eGFR_synthetic_20000.csv')
    input_text=rand_df['input_text']
    ############################
    #table_name="20241029_GEMINI_eGFR3"
    iterations=1
    upto=200000
    ############################
    resultsdf=pd.DataFrame(columns=['validnum','age', 'HDL', 'LDL', 'TG', 'HbA1c', 'Cr', 'Urate', 'CRP', 'SBP', 'DBP', 'BMI', 'WHratio', 'sex', 'HTN', 'DM', 'Dyslipidemia', 'Afib', 'CKD', 'CVDfamily', 'smoking','avg_gpt_score','c-index gpt','c-index acc/aha','c-index framingham'])


    for temper in [0.0]:
        templist=[]
        print('=========temperature========',temper)
        for i in range(iterations):
            newtablename=table_name + '_' + str(int(experimentnum)) + '_' + str(i)

            sql_statement="select * from ["+ newtablename + "]"
            conn = pymssql.connect(host=, database=, charset=) # enter pymssql connection info
            globals()['data{}'.format(i)] = pd.read_sql(sql=sql_statement, con=conn)


        for j in range(iterations):
            dat=globals()['data{}'.format(j)]

            eid=dat['eid']
            score_gpt=dat['score']
            category=dat['category']
            score_framingham=dat['framingham']
            score_acc_aha=dat['ACC_AHA']

            score_gpt2=[]

            for i in range(len(score_gpt)):
                try:
                    score_gpt2.append(float(re.findall("\d+[.]\d+[%]",score_gpt[i])[0].split('%')[0]))
                except:
                    try:
                        score_gpt2.append(float(re.findall("\d+[%]",score_gpt[i])[0].split('%')[0]))
                    except:
                        try:
                            score_gpt2.append(float(re.findall("\d+[.]\d+",score_gpt[i])[0]))
                        except:
                            try:
                                score_gpt2.append(float(re.findall("\d+",score_gpt[i])[-1]))
                            except:
                                score_gpt2.append(np.nan)
                                #score_gpt2.append(float(score_framingham[i]))
                                #print(score_gpt[i],'====')


        data_final=pd.DataFrame({'eid':data0['eid'],'score':score_gpt2})
        data_final['eid']=data_final['eid'].astype(int)
        print(len(data_final))
        data_final=data_final.dropna()
        data_final.reset_index(inplace=True,drop=True)


In [ ]:
data_final=pd.DataFrame({'eid':data0['eid'],'score':score_gpt2})
data_final['eid']=data_final['eid'].astype(int)
print(len(data_final))
data_final=data_final.dropna()
data_final.reset_index(inplace=True,drop=True)
data_final

In [ ]:
data_final=pd.DataFrame({'eid':data0['eid'],'score':score_gpt2})
data_final['eid']=data_final['eid'].astype(int)
print(len(data_final))
data_final=data_final.dropna()
data_final.reset_index(inplace=True,drop=True)

rand_df_temp=pd.read_csv('20241018_gptderived_FSRP_testdf.csv')
rand_df_temp=rand_df_temp.iloc[:20000]
rand_df=rand_df_temp.copy()
for i in range(9):
    rand_df=pd.concat([rand_df,rand_df_temp])
rand_df.reset_index(inplace=True,drop=True)
rand_df['eid']=range(200000)

#data_final_groupmean=data_final.groupby(['eid'],as_index=False).mean()
#df_merged=pd.merge(data_final_groupmean,rand_df,how='inner',on='eid')
df_merged=pd.merge(data_final,rand_df,how='inner',on='eid')
print(len(df_merged))

In [ ]:
df_merged.drop(['input_text'],axis=1,inplace=True)

In [ ]:
len(df_merged)

In [ ]:
smoking_ex=[]
smoking_current=[]
for i in range(len(df_merged)):
    if df_merged['smoking'][i]==2:
        smoking_current.append(1)
        smoking_ex.append(0)
    elif df_merged['smoking'][i]==1:
        smoking_current.append(0)
        smoking_ex.append(1)
    else:
        smoking_current.append(0)
        smoking_ex.append(0)
        
df_merged.drop(['smoking'],axis=1,inplace=True)
df_merged['smoking_ex']=smoking_ex
df_merged['smoking_current']=smoking_current

In [ ]:
from sklearn.linear_model import LinearRegression
columnslisttemp=list(df_merged.columns)
columnslisttemp=columnslisttemp[2:]

print(columnslisttemp)
#X=df_merged[['age','sex','HDL','LDL','TG','Chol','SBP','DBP','BMI','HTN','DM','smoking']]
X=df_merged[columnslisttemp]
y=df_merged['score']

line_fitter = LinearRegression()
line_fitter.fit(X, y)

In [ ]:
line_fitter.coef_

In [ ]:
X2=X.copy()
X2['y']=y
X2.to_csv('20241029_GEMINI_stroke_R.csv',index=False)

In [ ]:
len(X2)

# loading UKB

In [ ]:
#df_final_temp=pd.read_csv('20231122_df_final.csv')
df_final_temp=pd.read_csv('20241018_df_final_pastCVDnotexcluded.csv')

In [ ]:
columnslist=[]

columnslist.append('age')
columnslist.append('sex')
columnslist.append('total_chol')
columnslist.append('HDL')
columnslist.append('LDL')
columnslist.append('TG')
columnslist.append('SBP')
columnslist.append('DBP')
columnslist.append('med_final')
columnslist.append('DM_final')
columnslist.append('BMI_x')
columnslist.append('cig_final')
columnslist.append('E78_dyslipidemia_binary')
columnslist.append('I48_afib_binary')
columnslist.append('N18_CKD_binary')
columnslist.append('HbA1c')
columnslist.append('Creatinine')
columnslist.append('urate')
columnslist.append('C_reactive_ptn')
columnslist.append('CVD_firstdegree')
#columnslist.append('firstoccur_event')
columnslist.append('WHratio')

df_final_temp = df_final_temp.dropna(subset=columnslist)
df_final_temp.reset_index(inplace=True,drop=True)

In [ ]:
len(df_final_temp)

In [ ]:
df_final=df_final_temp
df_stroke=pd.read_csv('UKB_data/ukb_first_occur_for_regression_2.csv')
df_CHADVASC=pd.read_csv('UKB_data/ukb_first_occur_CHADVASC.csv')
df_stroke['eid']=df_stroke['eid'].astype(int)
df_CHADVASC['eid']=df_CHADVASC['eid'].astype(int)
df_CHADVASC.drop(['I63_cerebral_infarction','I64_stroke_not_specified'],axis=1,inplace=True)

In [ ]:
df_final

In [ ]:
np.sum(df_final['sex'])

In [ ]:
df_final=pd.merge(df_final, df_stroke, left_on='eid', right_on='eid', how='left')
df_final=pd.merge(df_final, df_CHADVASC, left_on='eid', right_on='eid', how='left')

In [ ]:
#df_final_afib=df_final[df_final['I48_afib_binary']==1]
df_final_afib=df_final.copy()
df_final_afib.reset_index(inplace=True,drop=True)
df_final_afib['assess_date']=pd.to_datetime(df_final_afib['assess_date'])

In [ ]:
CHADVASC_col=list(df_CHADVASC.columns)
stroke_col=list(df_stroke.columns)
CHADVASC_col=CHADVASC_col[1:]
stroke_col=stroke_col[1:]

for i in CHADVASC_col:
    df_final_afib[i]=pd.to_datetime(df_final_afib[i])
for i in stroke_col:
    df_final_afib[i]=pd.to_datetime(df_final_afib[i])

In [ ]:
for i in CHADVASC_col:
    templist=[]
    for j in range(len(df_final_afib)):
        if df_final_afib[i][j]<=df_final_afib['assess_date'][j]:
            templist.append(1)
        else:
            templist.append(0)
    df_final_afib[i+'_binary']=templist
    
for i in stroke_col:
    templist=[]
    for j in range(len(df_final_afib)):
        if df_final_afib[i][j]<=df_final_afib['assess_date'][j]:
            templist.append(1)
        else:
            templist.append(0)
    df_final_afib[i+'_binary']=templist
    
for i in stroke_col:
    templist=[]
    for j in range(len(df_final_afib)):
        if df_final_afib[i][j]>df_final_afib['assess_date'][j]:
            templist.append(1)
        else:
            templist.append(0)
    df_final_afib[i+'_binary_post']=templist

In [ ]:
len(df_final_afib)

In [ ]:
CHF_columns=['I50_heart_failure_binary']
PVD_columns=['I70_atherosclerosis_binary']

df_final_afib['CHD']=df_final_afib[CHF_columns].max(axis = 1, numeric_only = True)

df_final_afib['PVD']=df_final_afib[PVD_columns].max(axis = 1, numeric_only = True)

Stroke_post_columns=['I60_subarachnoid_haemorrhage_binary_post','I61_intracerebral_haemorrhage_binary_post','I62_other_haemorrhage_binary_post','I63_cerebral_infarction_binary_post','I64_stroke_not_specified_binary_post']

df_final_afib['stroke_post']=df_final_afib[Stroke_post_columns].max(axis = 1, numeric_only = True)

In [ ]:
for i in stroke_col:
    templist=[]
    for j in range(len(df_final_afib)):
        if df_final_afib[i][j]>df_final_afib['assess_date'][j] and df_final_afib[i][j]<=(df_final_afib['assess_date'][j]+ pd.DateOffset(years=1)):
            templist.append(1)
        else:
            templist.append(0)
    df_final_afib[i+'_binary_post_1year']=templist

In [ ]:
Stroke_post_1year_columns=['I60_subarachnoid_haemorrhage_binary_post_1year','I61_intracerebral_haemorrhage_binary_post_1year','I62_other_haemorrhage_binary_post_1year','I63_cerebral_infarction_binary_post_1year','I64_stroke_not_specified_binary_post_1year']

df_final_afib['stroke_post_1year']=df_final_afib[Stroke_post_1year_columns].max(axis = 1, numeric_only = True)

In [ ]:
print(len(df_final_afib[df_final_afib['smoking']==1]))
print(len(df_final_afib[df_final_afib['smoking']==2]))

In [ ]:
df_final_afib['firstoccur_date']=pd.to_datetime(df_final_afib['firstoccur_date'])
for i in ['firstoccur_date']:
    templist=[]
    for j in range(len(df_final_afib)):
        if df_final_afib[i][j]<=df_final_afib['assess_date'][j]:
            templist.append(1)
        else:
            templist.append(0)
    df_final_afib[i+'_binary']=templist

In [ ]:
import numpy as np

# Define coefficients for men and women
coefficients = {
    'women': {
        'age': 0.87938,
        'smk': 0.51127,
        'cvd': -0.03035,
        'af': 1.20720,
        'age65': 0.39796,
        'dm': 1.07111,
        'age65_dm': 0.06565,
        'hrx': 0.13085,
        'sbp_no_hrx': 0.11303,
        'sbp_hrx': 0.17234,
        'baseline_survival': 0.95911,
        'M': 6.6170719
    },
    'men': {
        'age': 0.49716,
        'smk': 0.47254,
        'cvd': 0.45341,
        'af': 0.08064,
        'age65': 0.45426,
        'dm': 1.35304,
        'age65_dm': 0.34385,
        'hrx': 0.27323,
        'sbp_no_hrx': 0.09793,
        'sbp_hrx': 0.27323,
        'baseline_survival': 0.94451,
        'M': 4.4227101
    }
}

# Define a function to calculate the linear combination (L) based on individual risk factors
def calculate_L(gender, age, smk, cvd, af, dm, hrx, sbp):
    coeff = coefficients[gender]
    L = (coeff['age'] * (age / 10) +
         coeff['smk'] * smk +
         coeff['cvd'] * cvd +
         coeff['af'] * af +
         coeff['age65'] * (age >= 65) +
         coeff['dm'] * (dm and age < 65) +
         coeff['age65_dm'] * (dm and age >= 65) +
         coeff['hrx'] * hrx +
         coeff['sbp_no_hrx'] * ((sbp - 120) / 10) * (1-hrx) +
         coeff['sbp_hrx'] * ((sbp - 120) / 10) * (hrx) )

    return L

# Define a function to calculate FSRP
def calculate_FSRP(gender, age, smk, cvd, af, dm, hrx, sbp):
    coeff = coefficients[gender]
    L = calculate_L(gender, age, smk, cvd, af, dm, hrx, sbp)
    S10 = coeff['baseline_survival']
    M = coeff['M']
    FSRP_10_year = 1 - S10**np.exp(L - M)
    return FSRP_10_year

# Example input (modify with actual patient data)
gender = 'women'  # Choose between 'women' and 'men'
age = 75
smk = 1  # 1 if current smoker, else 0
cvd = 1  # 1 if prevalent CVD, else 0
af = 0   # 1 if prevalent AF, else 0
dm = 0   # 1 if diabetes mellitus, else 0
hrx = 1  # 1 if on antihypertensive medication, else 0
sbp = 135  # Systolic blood pressure

fsrp_score = calculate_FSRP(gender, age, smk, cvd, af, dm, hrx, sbp)
fsrp_score*100

In [ ]:
FSRP=[]
for i in range(len(df_final_afib)):
    if df_final_afib['sex'][i]==0:
        gender2='women'
    else:
        gender2='men'
        
    age2=df_final_afib['age'][i]
    
    if df_final_afib['smoking'][i]==2:
        smk2=1
    else:
        smk2=0
        
    if df_final_afib['firstoccur_date_binary'][i]==1 or df_final_afib['I50_heart_failure_binary'][i]==1 or df_final_afib['I70_atherosclerosis_binary'][i]==1:
        cvd2=1
    else:
        cvd2=0
        
    if df_final_afib['I48_atrial_fibrillation_flutter_binary'][i]==1:
        af2=1
    else:
        af2=0
        
    if df_final_afib['DM_final'][i]==1:
        dm2=1
    else:
        dm2=0
        
    if df_final_afib['med_final'][i]==1:
        hrx2=1
    else:
        hrx2=0
        
    sbp2=df_final_afib['SBP'][i]
    
    fsrp_score = calculate_FSRP(gender2, age2, smk2, cvd2, af2, dm2, hrx2, sbp2)
    
    FSRP.append(fsrp_score)


In [ ]:
df_final_afib['FSRP']=FSRP
df_final_afib['FSRP']=df_final_afib['FSRP']*100

In [ ]:
plt.hist(df_final_afib['FSRP'],bins=20)

In [ ]:
df_final_afib3=df_final_afib[df_final_afib['I60_subarachnoid_haemorrhage_binary']==0]
df_final_afib3=df_final_afib3[df_final_afib3['I61_intracerebral_haemorrhage_binary']==0]
df_final_afib3=df_final_afib3[df_final_afib3['I62_other_haemorrhage_binary']==0]
df_final_afib3=df_final_afib3[df_final_afib3['I63_cerebral_infarction_binary']==0]
df_final_afib3=df_final_afib3[df_final_afib3['I64_stroke_not_specified_binary']==0]

In [ ]:
LRcolumns=[]
LRcolumns.append('age')
LRcolumns.append('HDL')
LRcolumns.append('LDL')
LRcolumns.append('TG')
LRcolumns.append('HbA1c')
LRcolumns.append('SBP')
LRcolumns.append('DBP')
LRcolumns.append('BMI_x')
LRcolumns.append('sex')
LRcolumns.append('med_final')
LRcolumns.append('DM_final')
LRcolumns.append('E78_dyslipidemia_binary')
LRcolumns.append('I48_afib_binary')
LRcolumns.append('N18_CKD_binary')
LRcolumns.append('I50_heart_failure_binary')
LRcolumns.append('firstoccur_date_binary')
LRcolumns.append('I70_atherosclerosis_binary')
LRcolumns.append('CVD_firstdegree')
LRcolumns.append('cig_final')

LRcolumnsclean=[]
LRcolumnsclean.append('age')
LRcolumnsclean.append('HDL')
LRcolumnsclean.append('LDL')
LRcolumnsclean.append('TG')
LRcolumnsclean.append('HbA1c')
LRcolumnsclean.append('SBP')
LRcolumnsclean.append('DBP')
LRcolumnsclean.append('BMI')
LRcolumnsclean.append('sex')
LRcolumnsclean.append('HTN')
LRcolumnsclean.append('DM')
LRcolumnsclean.append('Dyslipidemia')
LRcolumnsclean.append('Afib')
LRcolumnsclean.append('CKD')
LRcolumnsclean.append('CHF')
LRcolumnsclean.append('CAD')
LRcolumnsclean.append('PVD')
LRcolumnsclean.append('CVDfamily')
LRcolumnsclean.append('smoking')

In [ ]:
X=df_final_afib3[LRcolumns]
X.columns=LRcolumnsclean
X.reset_index(inplace=True,drop=True)

In [ ]:
smoking_ex=[]
smoking_current=[]
for i in range(len(X)):
    if X['smoking'][i]==2:
        smoking_current.append(1)
        smoking_ex.append(0)
    elif X['smoking'][i]==1:
        smoking_current.append(0)
        smoking_ex.append(1)
    else:
        smoking_current.append(0)
        smoking_ex.append(0)
        
X.drop(['smoking'],axis=1,inplace=True)
X['smoking_ex']=smoking_ex
X['smoking_current']=smoking_current

In [ ]:
y=line_fitter.predict(X)

In [ ]:
df_final_afib3['y']=y

In [ ]:
df_final_afib3.to_csv('20241030_GEMINI_df_final_afib_FSRP.csv',index=False)

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr,tpr,thres=roc_curve(df_final_afib3['stroke_post'],df_final_afib3['FSRP'])
auc(fpr, tpr)
print(auc(fpr, tpr))
print(np.mean(df_final_afib3['FSRP']))
print(np.mean(df_final_afib3['stroke_post']))
print(np.sum(df_final_afib3['stroke_post']))
print(len(df_final_afib3))

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr,tpr,thres=roc_curve(df_final_afib3['stroke_post'],df_final_afib3['y'])
auc(fpr, tpr)
print(auc(fpr, tpr))
print(np.mean(df_final_afib3['y']))
print(np.mean(df_final_afib3['stroke_post']))
print(np.sum(df_final_afib3['stroke_post']))
print(len(df_final_afib3))

In [ ]:
df_final_afib3.reset_index(inplace=True,drop=True)

In [ ]:
df_final_afib3['first_stroke_diagnosis_time'] = df_final_afib3[['I60_subarachnoid_haemorrhage', 'I61_intracerebral_haemorrhage', 
                                 'I62_other_haemorrhage', 'I63_cerebral_infarction', 
                                 'I64_stroke_not_specified']].min(axis=1)

In [ ]:
df_final_afib3['assess_date'] = pd.to_datetime(df_final_afib3['assess_date'], errors='coerce')

In [ ]:
df_final_afib3['first_stroke_diagnosis_time'] = df_final_afib3['first_stroke_diagnosis_time'].fillna(pd.Timestamp('2021-10-25 00:00:00'))


In [ ]:
df_final_afib3['days_difference'] = (df_final_afib3['first_stroke_diagnosis_time'] - df_final_afib3['assess_date']).dt.days

In [ ]:
from lifelines.utils import concordance_index

In [ ]:
concordance_index(df_final_afib3['days_difference'], -df_final_afib3['y'], df_final_afib3['stroke_post'])

In [ ]:
concordance_index(df_final_afib3['days_difference'], -df_final_afib3['FSRP'], df_final_afib3['stroke_post'])

In [ ]:
CINDEX_cols=['Cindex_LLM','Cindex_FSRP']
rows_list=[]

for iiii in range(2000):
    df_final_resample=df_final_afib3.sample(frac=1, replace=True)


    cindex_LLM=concordance_index(df_final_resample['days_difference'], -df_final_resample['y'], df_final_resample['stroke_post'])
    cindex_fsrp=concordance_index(df_final_resample['days_difference'], -df_final_resample['FSRP'], df_final_resample['stroke_post'])
    
    row = [
        cindex_LLM,
        cindex_fsrp
    ]
    rows_list.append(row)
    if iiii%100==0:
        print(iiii)
CINDEX_df = pd.DataFrame(rows_list, columns=CINDEX_cols)    
CINDEX_df.to_csv('20241030_GEMINI_stroke_CINDEX_bootstrap.csv',index=False)

In [ ]:
CINDEX_df['diff']=CINDEX_df['Cindex_LLM']-CINDEX_df['Cindex_FSRP']

In [ ]:
print(np.percentile(CINDEX_df['diff'],2.5))
print(np.percentile(CINDEX_df['diff'],97.5))

In [ ]:
print(np.percentile(CINDEX_df['Cindex_LLM'],2.5))
print(np.percentile(CINDEX_df['Cindex_LLM'],97.5))

In [ ]:
print(np.percentile(CINDEX_df['Cindex_FSRP'],2.5))
print(np.percentile(CINDEX_df['Cindex_FSRP'],97.5))